In [2]:
#basics
import pandas as pd 
import numpy as np
import os
import sys

#parser
import urllib
from bs4 import BeautifulSoup

#viz
import matplotlib.pyplot as plt

#NLP
import string
from autocorrect import spell
from nltk.corpus import wordnet

In [3]:
# 有效代码
cwd=os.getcwd()
fd_str=os.path.join(cwd,"input2\\gap-html")
print(fd_str)
# Get all dirs
dirnames = [name for name in os.listdir(fd_str)
        if os.path.isdir(os.path.join(fd_str, name))]
print(dirnames)

# Get all regular files
allfiles_names=[]
for dirname in dirnames:
    dirpath=os.path.join(fd_str,dirname)
    names = [os.path.join(dirpath, name) for name in os.listdir(dirpath) 
            if name.endswith('.html') and os.path.isfile(os.path.join(dirpath, name))]
    allfiles_names.append(names)  #二维列表
print(len(allfiles_names)) 
# print(allfiles_names[0])

D:\Code\Jupyter Notebook\COMP6237_Data_Mining\input2\gap-html
['gap_-C0BAAAAQAAJ', 'gap_2X5KAAAAYAAJ', 'gap_9ksIAAAAQAAJ', 'gap_aLcWAAAAQAAJ', 'gap_Bdw_AAAAYAAJ', 'gap_CnnUAAAAMAAJ', 'gap_CSEUAAAAYAAJ', 'gap_DhULAAAAYAAJ', 'gap_dIkBAAAAQAAJ', 'gap_DqQNAAAAYAAJ', 'gap_fnAMAAAAYAAJ', 'gap_GIt0HMhqjRgC', 'gap_IlUMAQAAMAAJ', 'gap_MEoWAAAAYAAJ', 'gap_m_6B1DkImIoC', 'gap_ogsNAAAAIAAJ', 'gap_pX5KAAAAYAAJ', 'gap_RqMNAAAAYAAJ', 'gap_TgpMAAAAYAAJ', 'gap_udEIAAAAQAAJ', 'gap_VPENAAAAQAAJ', 'gap_WORMAAAAYAAJ', 'gap_XmqHlMECi6kC', 'gap_y-AvAAAAYAAJ']
24


In [4]:
# 有效代码
def read_file(file_path):
#     with open(file_path, 'rt',encoding='latin-1') as f:
    with open(file_path, 'rt',encoding='utf-8') as f:
        data = f.read()
    return data

In [5]:
# 有效代码
def write_file(file_path,text):
    with open(file_path,'w',encoding='utf-8') as f:
        f.write(text)

In [6]:
# 有效代码
def extract_text_by_bs(doc_text):
    """提取html页面的所有文本信息。
    参考：
    http://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python
    """
    title, text = '', ''
    soup = BeautifulSoup(doc_text, 'lxml')
    try:
        for script in soup(["script", "style"]):
            script.extract()
    except Exception as error:
        print(error)
        pass
    else:
        try:
            # get text
            title = soup.title.string
        except Exception as error:
            print(error)
            pass
        text = soup.get_text()
        print(text)
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        print("Lines:")
#         lines = (print(line.strip()) for line in text.splitlines())
        for line in text.splitlines():
            print(line+"\r\n")
        
        print("Chunks:")
        for line in lines:
            for phrase in line.split("  "):
                print(phrase)
    return title, text

In [7]:
def extract_text_by_bs2(doc_text):
    title, text = '', ''
    soup = BeautifulSoup(doc_text, 'lxml')
    spans_lines=soup.find_all("span","ocr_line")
    spans_cinfo=soup.find_all("span","ocr_cinfo")
#     print(spans_line)
#     for span_line in spans_lines:
#         print(span_line.get_text())
    text_list=[span_line.get_text() for span_line in spans_lines]
    text='\r\n'.join(text_list)
    return text

In [68]:
'''
将一行的末单词和下一行的首单词能组成词的放在了下一行首，麻烦，放在该行的末尾简单
'''
def extract_text_by_bs3(doc_text):
    title, text = '', ''
    soup = BeautifulSoup(doc_text, 'lxml')
    spans_lines=soup.find_all("span","ocr_line")
#     print(spans_lines)
    if(spans_lines==[]):
        return text
    
    prev_word=''
    sent_prev_line=''
    spans_cinfo_text_list_prev=[]
    i=0
    for span_line in spans_lines:
#         print(span_line.get_text())
        spans_cinfos=span_line.find_all("span","ocr_cinfo")
#         print(spans_cinfos)
        if(spans_cinfos==[]):
            continue
            
        spans_cinfo_text_list=[span.get_text() for span in spans_cinfos]
#         print(str(i)+':',spans_cinfo_text_list)

        spans_cinfo_text_list1=spans_cinfo_text_list[1:len(spans_cinfo_text_list)-1] #去除第一个和最后一个词
   
        sent_line=' '.join(spans_cinfo_text_list1)
        first_word=spans_cinfo_text_list[0]
        combine_word=prev_word+first_word
        if(wordnet.synsets(combine_word)!=[] and wordnet.synsets(first_word)==[]): #一行的末单词和下一行的首单词构成一个词
#       下一行的首单词不是一个词   
            sent_line=combine_word+' '+sent_line
        else:
            sent_line=first_word+' '+sent_line
            if(i>0): #第一行不用
                if(len(spans_cinfo_text_list_prev)>1):
                    sent_prev_line+=' '+spans_cinfo_text_list_prev[len(spans_cinfo_text_list_prev)-1]  #前一行加入最后一个词
#                 print(str(i)+'prev:',sent_prev_line)
        
#         print(str(i)+'cur:',sent_line)
        
        text+=sent_prev_line+'\r'+'\n'  
#         print(text)
        
        last_word=spans_cinfo_text_list[len(spans_cinfo_text_list)-1]
        prev_word=last_word
        sent_prev_line=sent_line
        spans_cinfo_text_list_prev=spans_cinfo_text_list
       
        i=i+1
        
    if(len(spans_cinfo_text_list_prev)>1):
        text+=sent_line+' '+spans_cinfo_text_list[len(spans_cinfo_text_list)-1]+'\r'+'\n' 
    return text

In [34]:
name='gap_9ksIAAAAQAAJ\\00000009.html'
file_path = os.path.join(fd_str, name)
print(file_path)
file_data=read_file(file_path) 
# titel,text=extract_text_by_bs(file_data)
# print(text)

D:\Code\Jupyter Notebook\COMP6237_Data_Mining\input2\gap-html\gap_9ksIAAAAQAAJ\00000009.html


In [17]:
text=extract_text_by_bs2(file_data)
# print(text)

In [65]:
name='gap_9ksIAAAAQAAJ\\00000009.html'
file_path = os.path.join(fd_str, name)
print(file_path)
file_data=read_file(file_path) 
text=extract_text_by_bs3(file_data)
print(text)

D:\Code\Jupyter Notebook\COMP6237_Data_Mining\input2\gap-html\gap_9ksIAAAAQAAJ\00000009.html

Contents of the Second Volume.
CONTENTS of Book V,
YEAR X. The truce endeth. Cleon sent commander Into
Thrace; his proceedings there. The battle of Arophipolis,
in which Brasidas and Cleon are killed. A general peace, Ailed
the Nician. An alliance, offensive and defensive, between the
Athenians and Lacedæmonians. — — P, I
Year Xs. The peace merely nominal; and Thucydides proceedeth
in his history of the Peloponnesian war. The Corinthians
practise against the Lacedæmonians. An Argive league. No
confidence between the principal States. A train of negotiations,
A separate alliance between the Lacedæmonians and Bœotians,
contrary to article. Panactum demolished. — P. 23
Year XII, The demolition of Panactum and the separate alliance
highly resented'at Athens. Many are scheming a rupture, but
especially Alcibiades. By his means a negotiation is brought on
at Athens, and an alliance formed with the A

In [67]:
# 有效代码
fd_store=os.path.join(cwd,"output3")
if(not os.path.exists(fd_store)):
    os.mkdir(fd_store)
for i in range(len(allfiles_names)):
# for i in range(2):
    print("i:"+str(i))
    text_combine=''
    j=0
    for file_path in allfiles_names[i]:
        path_list=file_path.split('\\')
        dir_name=path_list[len(path_list)-2]
        fd_store_dir=os.path.join(fd_store, dir_name)
        if(not os.path.exists(fd_store_dir)):
            os.mkdir(fd_store_dir)
        name=path_list[len(path_list)-1]
        name2=name.split(".")[0]
        file_store_path=os.path.join(fd_store_dir,name2+".txt")
        
        file_data=read_file(file_path) 
        text=extract_text_by_bs3(file_data)
        write_file(file_store_path,text)  #every file parsed from html files save to the folder
        
        text_combine = text_combine+" "+text.strip()
        
        if(j<3):
            print(str(j)+","+text_combine)
        j=j+1
        
    file_com_path=os.path.join(fd_store,dir_name+".txt") #
    write_file(file_com_path,text_combine)

i:0
0, 
1,  800020427L 
  
► 
  
ZOI5  ZOI5
2,  800020427L 
  
► 
  
ZOI5  ZOI5 DICTIONARY 
GREEK AND ROMAN GEOGRAPHY.
  
ns
i:1
0, 
1,  CReSlorecl  through
a arant in
from 
( 
The Cartwright Foundation 5
( 
PRINCETON 
UNIVERSITY 
K LIBRARY J
2,  CReSlorecl  through
a arant in
from 
( 
The Cartwright Foundation 5
( 
PRINCETON 
UNIVERSITY 
K LIBRARY J 
i:2
0, 
1,  
2,   
i:3
0, 
1,  
2,   4 s3<? • (
i:4
0, University of Virginia Library
DG207 L5 B3 1797 V.1
ALD The history ot Rome.__
II ii II ill ii mi IJIIM imuii
UX DD1 132 Sit,
1, University of Virginia Library
DG207 L5 B3 1797 V.1
ALD The history ot Rome.__
II ii II ill ii mi IJIIM imuii
UX DD1 132 Sit, 
2, University of Virginia Library
DG207 L5 B3 1797 V.1
ALD The history ot Rome.__
II ii II ill ii mi IJIIM imuii
UX DD1 132 Sit,  
i:5
0, 
1,  Univcr?  '-
.-..3 
General Library System
University of Wisconsin - Madison
728 State Street
Madison, Wl 53706-1494
U.S.A.
2,  Univcr?  '-
.-..3 
General Library System
University of Wisconsin

In [64]:
name='gap_2X5KAAAAYAAJ\\00000002.html' 
file_path = os.path.join(fd_str, name)
print(file_path)
file_data=read_file(file_path) 
text=extract_text_by_bs3(file_data)
print(text)

D:\Code\Jupyter Notebook\COMP6237_Data_Mining\input2\gap-html\gap_2X5KAAAAYAAJ\00000002.html

CReSlorecl  through
a arant in
from 
( 
The Cartwright Foundation 5
( 
PRINCETON 
UNIVERSITY 
K LIBRARY J

